In [97]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Data formating

In [2]:
data = pd.read_csv(r"Z:\TC_Sydney_Uni_research_extract_202410.csv")
data = data[data['ROUTE'] == 'LGHTRL'].reset_index(drop=True)  # Filter by light train
data = data[['ORIGIN_DATE', 'ORIGIN_STOP', 'ORIGIN_STOP_XY',
             'DESTINATION_DATE', 'DESTINATION_STOP', 'DESTINATION_STOP_XY']]  # Filter columns by relevance

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136346 entries, 0 to 136345
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ORIGIN_DATE          136346 non-null  object
 1   ORIGIN_STOP          136346 non-null  object
 2   ORIGIN_STOP_XY       136346 non-null  object
 3   DESTINATION_DATE     136346 non-null  object
 4   DESTINATION_STOP     136346 non-null  object
 5   DESTINATION_STOP_XY  136346 non-null  object
dtypes: object(6)
memory usage: 6.2+ MB


In [3]:
# Delete the distinction between platforms in the origin and destination

def remove_platform(text):
    if 'Platform' in text:
        return text.split('Platform')[0].strip()
    return text

data['ORIGIN_STOP'] = data['ORIGIN_STOP'].map(remove_platform)
data['DESTINATION_STOP'] = data['DESTINATION_STOP'].map(remove_platform)

# Change the stops for an ID
stops_id = {'8100: Gungahlin Place': 0,
            '8101: Gungahlin Place': 0,
            '8104: Manning Clark Crescent': 1,
            '8105: Manning Clark Crescent': 1,
            '8106: Mapleton Avenue': 2,
            '8107: Mapleton Avenue': 2,
            '8108: Nullarbor Avenue': 3,
            '8109: Nullarbor Avenue': 3,
            '8110: Well Station Drive': 4,
            '8111: Well Station Drive': 4,
            '8114: EPIC and Racecourse': 5,
            '8116: Phillip Avenue': 6,
            '8117: Phillip Avenue': 6,
            '8118: Swinden Street': 7,
            '8119: Swinden Street': 7,
            '8120: Dickson': 8,
            '8121: Dickson': 8,
            '8122: Macarthur Avenue': 9,
            '8123: Macarthur Avenue': 9,
            '8124: Ipima Street': 10,
            '8125: Ipima Street': 10,
            '8126: Elouera Street': 11,
            '8127: Elouera Street': 11,
            '8128: Alinga Street': 12,
            '8129: Alinga Street': 12
            }

data['ORIGIN_STOP'] = data['ORIGIN_STOP'].replace(stops_id)
data['DESTINATION_STOP'] = data['DESTINATION_STOP'].replace(stops_id)

In [4]:
# Average demand between 7am and 8am
minimal_time = 8
maximal_time = 9

# Selected date
selected_date = pd.to_datetime('2024-09-18').date()

# To datetime
data['ORIGIN_DATE'] = pd.to_datetime(data['ORIGIN_DATE'], format='%Y-%m-%d %H:%M:%S')
data['DESTINATION_DATE'] = pd.to_datetime(data['DESTINATION_DATE'], format='%Y-%m-%d %H:%M:%S')
# Calculate only the time
data['ORIGIN_TIME'] = data['ORIGIN_DATE'].dt.hour * 60 + data['ORIGIN_DATE'].dt.minute
data['DESTINATION_TIME'] = data['DESTINATION_DATE'].dt.hour * 60 + data['DESTINATION_DATE'].dt.minute
# Filter by date
data['ORIGIN_DATE_DATE'] = data['ORIGIN_DATE'].dt.date
data = data[data['ORIGIN_DATE_DATE'] == selected_date]
# Filter by time
data = data[(data['ORIGIN_TIME'] >= minimal_time*60) & (data['ORIGIN_TIME'] < maximal_time*60)]
# Group demand
demand = data.groupby(['ORIGIN_STOP', 'DESTINATION_STOP']).size().reset_index(name='DEMAND')

demand.head()

,ORIGIN_STOP,DESTINATION_STOP,DEMAND
0,0,0,11
1,0,1,1
2,0,2,10
3,0,3,41
4,0,4,4


In [5]:
# Give the origin-destination matrix shape
demand = demand.pivot(columns='DESTINATION_STOP', index='ORIGIN_STOP', values='DEMAND')
# Make the diagonal zero
for i in range(demand.shape[0]):
    demand.loc[i,i] = 0
# Fill with zero
demand.fillna(0, inplace=True)
# Sort columns
demand = demand[sorted(demand.columns)]

demand

DESTINATION_STOP,0,1,2,3,4,5,6,7,8,9,10,11,12
ORIGIN_STOP,,,,,,,,,,,,,
0,0.0,1.0,10.0,41.0,4.0,0.0,4.0,9.0,84.0,5.0,15.0,21.0,394.0
1,9.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,14.0,0.0,2.0,8.0,49.0
2,34.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,32.0,3.0,9.0,4.0,115.0
3,19.0,0.0,0.0,0.0,3.0,0.0,2.0,1.0,14.0,5.0,1.0,10.0,92.0
4,8.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,14.0,0.0,4.0,4.0,102.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,15.0
6,4.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,56.0
7,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,64.0
8,16.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,5.0,14.0,204.0


In [6]:
demand.loc[:i, i]

ORIGIN_STOP
0     394.0
1      49.0
2     115.0
3      92.0
4     102.0
5      15.0
6      56.0
7      64.0
8     204.0
9     154.0
10     66.0
11     11.0
12      0.0
Name: 12, dtype: float64

In [7]:
# To fabricate the travel times with google maps
travel_time = np.zeros((13,13))
travel_time[0,1] = 2
travel_time[1,2] = 1
travel_time[2,3] = 2
travel_time[3,4] = 2
travel_time[4,5] = 4
travel_time[5,6] = 2
travel_time[6,7] = 2
travel_time[7,8] = 2
travel_time[8,9] = 2
travel_time[9,10] = 2
travel_time[10,11] = 1
travel_time[11,12] = 2

for i in range(13):
    for j in range(i+1, 13):
        travel_time[i,j] = travel_time[i,j-1] + travel_time[j-1,j]
        travel_time[j,i] = travel_time[i,j]

travel_time = pd.DataFrame(travel_time)
travel_time

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,2.0,3.0,5.0,7.0,11.0,13.0,15.0,17.0,19.0,21.0,22.0,24.0
1,2.0,0.0,1.0,3.0,5.0,9.0,11.0,13.0,15.0,17.0,19.0,20.0,22.0
2,3.0,1.0,0.0,2.0,4.0,8.0,10.0,12.0,14.0,16.0,18.0,19.0,21.0
3,5.0,3.0,2.0,0.0,2.0,6.0,8.0,10.0,12.0,14.0,16.0,17.0,19.0
4,7.0,5.0,4.0,2.0,0.0,4.0,6.0,8.0,10.0,12.0,14.0,15.0,17.0
5,11.0,9.0,8.0,6.0,4.0,0.0,2.0,4.0,6.0,8.0,10.0,11.0,13.0
6,13.0,11.0,10.0,8.0,6.0,2.0,0.0,2.0,4.0,6.0,8.0,9.0,11.0
7,15.0,13.0,12.0,10.0,8.0,4.0,2.0,0.0,2.0,4.0,6.0,7.0,9.0
8,17.0,15.0,14.0,12.0,10.0,6.0,4.0,2.0,0.0,2.0,4.0,5.0,7.0
9,19.0,17.0,16.0,14.0,12.0,8.0,6.0,4.0,2.0,0.0,2.0,3.0,5.0


In [8]:
# # Calculate travel times
# data['TRAVEL_TIME_MINUTES'] = data['DESTINATION_TIME'] - data['ORIGIN_TIME']
# travel_time = data[['ORIGIN_STOP', 'DESTINATION_STOP', 'TRAVEL_TIME_MINUTES']].groupby(['ORIGIN_STOP', 'DESTINATION_STOP']).mean().reset_index()
# travel_time = travel_time.pivot(columns='DESTINATION_STOP', index='ORIGIN_STOP', values='TRAVEL_TIME_MINUTES')
#
# # Make the diagonal zero
# for i in range(travel_time.shape[0]):
#     travel_time.loc[i,i] = 0
# # Fill with zero
# travel_time.fillna(0, inplace=True)
# # Sort columns
# travel_time = travel_time[sorted(travel_time.columns)]
#
# travel_time

# DI implementation

In [9]:
def transfers_calculate(demand: pd.DataFrame) -> list[int]:
    """
    For each stop, calculate the potential transfers adding the two directions.
    """
    transfers_list = []
    for i in demand.index:
        # The transfers are the passengers traveling from before i to after i.
        trans = demand.loc[:i-1, i+1:].values.sum()
        # Also, the transfers are the passengers traveling from after i to before i.
        trans += demand.loc[i+1:, :i-1].values.sum()
        transfers_list.append(trans)

    return transfers_list

def max_flow_calculate(demand: pd.DataFrame) -> list[tuple[int, int]]:
    """
    For each stop, calculate the maximum flow before and after, including both directions.
    """
    # Calculate flow both directions in each segment
    flow_left = []
    flow_right = []
    for i in range(len(demand.index)-1):
        # Passengers traveling from before i to after i.
        flow_l = demand.loc[:i, i+1:].values.sum()
        flow_left.append(flow_l)
        # Passengers traveling from after i to before i.
        flow_r = demand.loc[i+1:, :i].values.sum()
        flow_right.append(flow_r)

    # Maximum flow by arc
    flow_max = [max(flow_left[i], flow_right[i]) for i in range(len(flow_left))]
    flow_max = np.array(flow_max)

    # Maximum flow both side division
    flow_max_division = [0]*len(demand.index)
    for i in range(1, len(demand.index)-1):
        flow_max_division[i] = (flow_max[:i].max(), flow_max[i:].max())

    return flow_max_division

def length_term_calculate(travel_time: pd.DataFrame, flow_max_division: list[tuple[int, int]]) -> list[int]:
    """
    For each stop, calculate |l_i| + |l_i|/|l|
    """
    length = [0]*len(travel_time.index)
    l = travel_time.loc[0,:].values.sum()
    for i in range(1, len(travel_time.index)-1):
        # The maximum flow segment is in the right
        if flow_max_division[i][0] < flow_max_division[i][1]:
            li = travel_time.loc[0,i]
        # The maximum flow segment is in the left
        else:
            li = travel_time.loc[i,travel_time.index[-1]]
        # Calculate and salve
        length[i] = li + li/l

    return length

def di_calculation(demand: pd.DataFrame, travel_time: pd.DataFrame, d1: float, d2: float) -> list[int]:
    """
    Divisibility index for each stop.
    """
    # Calculate the terms
    transfers = transfers_calculate(demand)
    flow_max = max_flow_calculate(demand)
    length = length_term_calculate(travel_time, flow_max)

    # Calculate the DI
    total_demand = demand.values.sum()
    divisibility = [0]*len(demand.index)
    for i in range(1, len(travel_time.index)-1):
        term1 = 1 - d1*transfers[i]/total_demand
        term2 = abs(flow_max[i][0] - flow_max[i][1])
        term3 = 1 + d2*length[i]
        divisibility[i] = term1 + term2 + term3

    return divisibility

In [10]:
transfers_calculate(demand)

[np.float64(0.0),
 np.float64(706.0),
 np.float64(741.0),
 np.float64(844.0),
 np.float64(963.0),
 np.float64(1097.0),
 np.float64(1100.0),
 np.float64(1162.0),
 np.float64(1056.0),
 np.float64(1324.0),
 np.float64(1446.0),
 np.float64(1451.0),
 np.float64(0.0)]

In [11]:
max_flow_calculate(demand)

[0,
 (np.float64(588.0), np.float64(1381.0)),
 (np.float64(666.0), np.float64(1381.0)),
 (np.float64(820.0), np.float64(1381.0)),
 (np.float64(902.0), np.float64(1381.0)),
 (np.float64(1021.0), np.float64(1381.0)),
 (np.float64(1039.0), np.float64(1381.0)),
 (np.float64(1096.0), np.float64(1381.0)),
 (np.float64(1153.0), np.float64(1381.0)),
 (np.float64(1216.0), np.float64(1381.0)),
 (np.float64(1353.0), np.float64(1381.0)),
 (np.float64(1381.0), np.float64(1322.0)),
 0]

In [12]:
length_term_calculate(travel_time, max_flow_calculate(demand))

[0,
 np.float64(2.0125786163522013),
 np.float64(3.018867924528302),
 np.float64(5.031446540880503),
 np.float64(7.044025157232705),
 np.float64(11.069182389937106),
 np.float64(13.081761006289309),
 np.float64(15.09433962264151),
 np.float64(17.10691823899371),
 np.float64(19.11949685534591),
 np.float64(21.132075471698112),
 np.float64(2.0125786163522013),
 0]

In [13]:
di_calculation(demand, travel_time, 0.9, 0.08)

[0,
 np.float64(794.8409055336407),
 np.float64(716.9055396606625),
 np.float64(563.0198456980816),
 np.float64(481.1268973274401),
 np.float64(362.38815423855016),
 np.float64(344.547800326347),
 np.float64(287.6806957844209),
 np.float64(230.88976252712956),
 np.float64(167.92925748142517),
 np.float64(31.034948909272373),
 np.float64(60.50312215832581),
 0]

In [14]:
# flujo hacia la derecha
subidas = []
for i in demand.index:
    subidas.append(demand.loc[i, i:].values.sum())

bajadas = []
for i in demand.index:
    bajadas.append(demand.loc[:i, i].values.sum())

flujos_explicito = pd.DataFrame({'subidas': subidas, 'bajadas':bajadas})
flujos_explicito['subidas acumuladas'] = flujos_explicito['subidas'].cumsum()
flujos_explicito['bajadas acumuladas'] = flujos_explicito['bajadas'].cumsum()
flujos_explicito['capacidad'] = flujos_explicito['subidas acumuladas']-flujos_explicito['bajadas acumuladas']
flujos_explicito

,subidas,bajadas,subidas acumuladas,bajadas acumuladas,capacidad
0,588.0,0.0,588.0,0.0,588.0
1,79.0,1.0,667.0,1.0,666.0
2,165.0,11.0,832.0,12.0,820.0
3,128.0,46.0,960.0,58.0,902.0
4,127.0,8.0,1087.0,66.0,1021.0
5,18.0,0.0,1105.0,66.0,1039.0
6,64.0,7.0,1169.0,73.0,1096.0
7,70.0,13.0,1239.0,86.0,1153.0
8,226.0,163.0,1465.0,249.0,1216.0
9,156.0,19.0,1621.0,268.0,1353.0


In [15]:
# flujo hacia la izquierda
subidas = []
for i in demand.index:
    subidas.append(demand.loc[i, :i].values.sum())

bajadas = []
for i in demand.index:
    bajadas.append(demand.loc[i:, i].values.sum())

flujos_explicito = pd.DataFrame({'subidas': subidas, 'bajadas':bajadas})
flujos_explicito['subidas acumuladas'] = flujos_explicito['subidas'][::-1].cumsum()
flujos_explicito['bajadas acumuladas'] = flujos_explicito['bajadas'][::-1].cumsum()
flujos_explicito['capacidad'] = flujos_explicito['subidas acumuladas']-flujos_explicito['bajadas acumuladas']
flujos_explicito

,subidas,bajadas,subidas acumuladas,bajadas acumuladas,capacidad
0,0.0,128.0,286.0,286.0,0.0
1,9.0,1.0,286.0,158.0,128.0
2,34.0,3.0,277.0,157.0,120.0
3,19.0,7.0,243.0,154.0,89.0
4,8.0,7.0,224.0,147.0,77.0
5,0.0,0.0,216.0,140.0,76.0
6,8.0,15.0,216.0,140.0,76.0
7,4.0,6.0,208.0,125.0,83.0
8,19.0,75.0,204.0,119.0,85.0
9,14.0,14.0,185.0,44.0,141.0


# Value of the consumed resources

In [98]:
def cost_users(demand: pd.DataFrame,
               travel_time: pd.DataFrame,
               frequency: list[int],
               board_alight_time: float,
               piv: float,
               pw: float,
               pr: float,
               divided_nodes: list[int]) -> float:
    """
    Total users cost, given the divisions of the corridor.
    """
    # Transfer cost
    transfers = np.array(transfers_calculate(demand))
    transfers = [transfers[i] if i in divided_nodes
                 else 0
                 for i in range(len(demand.index))]
    transfers_totales = sum(transfers)

    # Waiting time cost
    waiting_time = 0
    for i in divided_nodes:
        line = find_line(divided_nodes=divided_nodes, stop=i)
        boarding_due_transfers = transfers[i]/(2*frequency[line])
        first_boarding = demand.loc[i, :].sum()/(2*frequency[line])
        waiting_time += boarding_due_transfers + first_boarding

    # Travel time cost
    stop_state_right, stop_state_left = board_alight_calculate(demand)
    total_travel_time = 0
    # In-vehicle moving time
    for i in demand.index:
        for j in demand.columns:
            total_travel_time += demand.loc[i,j]*travel_time.loc[i,j]
    # In-vehicle stopped time
    for i in demand.index:
        line = find_line(divided_nodes=divided_nodes, stop=i)
        # boarding and alighting passengers
        boarding = (stop_state_left.loc[i, 'subidas'] + stop_state_right.loc[i, 'subidas'])*board_alight_time/frequency[line]
        alight = (stop_state_left.loc[i, 'bajadas'] + stop_state_right.loc[i, 'bajadas'])*board_alight_time/frequency[line]
        # Passengers on board waiting
        stopped_passengers = demand.loc[:i-1, i+1:].values.sum() + demand.loc[i+1:, :i-1].values.sum()
        total_travel_time += stopped_passengers*(boarding+alight)
        # Passengers alight
        avr_alight_time = (stop_state_left.loc[i, 'bajadas'] + stop_state_right.loc[i, 'bajadas'])*board_alight_time/(2*frequency[line])
        total_travel_time += (stop_state_left.loc[i, 'bajadas'] + stop_state_right.loc[i, 'bajadas'])*avr_alight_time

    return total_travel_time*piv + waiting_time*pw + transfers_totales*pr

def board_alight_calculate(demand: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    It calculares the board and alight for each stop, for both directions.
    """

    # flow through the right
    subidas = []
    for i in demand.index:
        subidas.append(demand.loc[i, i:].values.sum())

    bajadas = []
    for i in demand.index:
        bajadas.append(demand.loc[:i, i].values.sum())

    flujos_explicito_right = pd.DataFrame({'subidas': subidas, 'bajadas':bajadas})

    # flow through the left
    subidas = []
    for i in demand.index:
        subidas.append(demand.loc[i, :i].values.sum())

    bajadas = []
    for i in demand.index:
        bajadas.append(demand.loc[i:, i].values.sum())

    flujos_explicito_left = pd.DataFrame({'subidas': subidas, 'bajadas':bajadas})

    return flujos_explicito_right, flujos_explicito_left

def find_line(divided_nodes: list[int], stop: int) -> int:
    """
    Given the nodes where the corridor is divided, it gives you the line where a given stop belongs. The lines are numbered from 0 and growing to the right.
    """
    if len(divided_nodes) == 0:
        return 0
    for i in range(len(divided_nodes)):
        if stop < divided_nodes[i]:
            return i
    return len(divided_nodes)

def cost_operators(demand: pd.DataFrame,
                   travel_time: pd.DataFrame,
                   frequency: list[int],
                   board_alight_time: float,
                   c0: float,
                   c1: float,
                   divided_nodes: list[int]) -> float:
    """
    It calculates the operator cost.
    """
    # Calculate flow both directions in each segment
    flow_left = []
    flow_right = []
    for i in range(len(demand.index)-1):
        # Passengers traveling from before i to after i.
        flow_l = demand.loc[:i, i+1:].values.sum()
        flow_left.append(flow_l)
        # Passengers traveling from after i to before i.
        flow_r = demand.loc[i+1:, :i].values.sum()
        flow_right.append(flow_r)
    # Maximum flow by arc
    flow_max = [max(flow_left[i], flow_right[i]) for i in range(len(flow_left))]
    flow_max = np.array(flow_max)

    # Calculate transfers
    transfers = np.array(transfers_calculate(demand))

    total_cost = 0
    # Calculate the cost for each line
    for i in range(len(frequency)):

        # Line terminal stops
        # Only one line
        if len(frequency) == 1:
            first_node = 0
            last_node = len(demand.index)-1
        # First (not only) line
        elif i == 0:
            first_node = 0
            last_node = divided_nodes[i]
        # Other than first
        else:
            first_node = divided_nodes[i-1]
            if i == len(frequency) - 1:
                last_node = demand.index[-1]
            else:
                last_node = divided_nodes[i]

        # Maximum flow in the line
        flow_max_line = flow_max[first_node:last_node].max()
        # Capacity
        capacity = flow_max_line/frequency[i]

        # Cycle time
        total_travel_time = travel_time.loc[first_node, last_node]
        stop_state_right, stop_state_left = board_alight_calculate(demand)
        boarding_time = (stop_state_left.loc[first_node+1: last_node, 'subidas'].values.sum() +
                         stop_state_right.loc[first_node: last_node-1, 'subidas'].values.sum())
        alight_time = (stop_state_left.loc[first_node: last_node-1, 'bajadas'].values.sum() +
                         stop_state_right.loc[first_node+1: last_node, 'bajadas'].values.sum())
        extra_boarding_time_transfers = transfers[first_node]
        cycle_time = total_travel_time + (boarding_time + alight_time + extra_boarding_time_transfers)*2*board_alight_time/frequency[i]

        # Fleet size
        fleet = cycle_time*frequency[i]

        # Add cost
        total_cost += fleet*(c0 + c1*capacity)

    return total_cost

def vrc(demand: pd.DataFrame,
        travel_time: pd.DataFrame,
        frequency: list[int],
        divided_nodes: list[int],
        piv: float,
        pw: float,
        pr: float,
        board_alight_time: float,
        c0: float,
        c1: float) -> float:
    """
    It calculates the VRC.
    """
    users = cost_users(demand=demand,
                       travel_time=travel_time,
                       frequency=frequency,
                       board_alight_time=board_alight_time,
                       piv=piv,
                       pw=pw,
                       pr=pr,
                       divided_nodes=divided_nodes)

    operators = cost_operators(demand=demand,
                               travel_time=travel_time,
                               frequency=frequency,
                               board_alight_time=board_alight_time,
                               c0 = c0,
                               c1 = c1,
                               divided_nodes=divided_nodes)

    return users + operators

def vrc_fix(demand: pd.DataFrame,
            travel_time: pd.DataFrame,
            piv: float,
            board_alight_time: float,
            c0: float,
            c1: float) -> float:
    """
    Calculates the VRC components that doesn't vary with the frequency or the line divisions.
    """

    # Users travel time in motion
    travel_time_motion = 0
    # In-vehicle moving time
    for i in demand.index:
        for j in demand.columns:
            travel_time_motion += demand.loc[i,j]*travel_time.loc[i,j]
    users_vrc_fix = travel_time_motion*piv

    # Operators
    cicle_time_motion = travel_time.loc[0, travel_time.index[-1]]
    total_demand = demand.values.sum()
    # Calculate flow both directions in each segment
    flow_left = []
    flow_right = []
    for i in range(len(demand.index)-1):
        # Passengers traveling from before i to after i.
        flow_l = demand.loc[:i, i+1:].values.sum()
        flow_left.append(flow_l)
        # Passengers traveling from after i to before i.
        flow_r = demand.loc[i+1:, :i].values.sum()
        flow_right.append(flow_r)
    # Maximum flow by arc
    flow_min = min(flow_left + flow_right)
    operators_vrc_fix = 2*board_alight_time*c0*total_demand + cicle_time_motion*flow_min*c1

    return users_vrc_fix + operators_vrc_fix

def vrc_optimization(demand: pd.DataFrame,
                     travel_time: pd.DataFrame,
                     divided_nodes: list[int],
                     piv: float,
                     pw: float,
                     pr: float,
                     board_alight_time: float,
                     c0: float,
                     c1: float) -> tuple[tuple[float], float]:
    """
    It calculates the optimal frequency to minimize the VRC.
    """

    def vrc_auxiliar(freq):
        vrc_value = vrc(demand=demand,
                        travel_time=travel_time,
                        frequency=freq,
                        divided_nodes=divided_nodes,
                        piv=piv,
                        pw=pw,
                        pr=pr,
                        board_alight_time=board_alight_time,
                        c0=c0,
                        c1=c1)
        return vrc_value

    # Minimization
    initial_freq = [30]*(len(divided_nodes)+1)
    result = minimize(vrc_auxiliar, initial_freq)

    optimal_freq = result.x
    optimal_vrc = result.fun

    return optimal_freq, optimal_vrc

# Numerical calculations

In [90]:
c0 = 8.61
c1 = 0.3
piv = 2.74
pw = 5.48
pr = 16*piv
t = 0.0416 / 60

In [95]:
# Fixed cost
vrc_fix_cost = vrc_fix(demand=demand,
                       travel_time=travel_time,
                       piv=piv,
                       board_alight_time=t,
                       c0=c0,
                       c1=c1)

# Complete line
vrc_complete_line = vrc(demand=demand,
                        travel_time=travel_time,
                        frequency=[12],
                        divided_nodes=[],
                        piv=piv,
                        pw=pw,
                        pr=pr,
                        board_alight_time=t,
                        c0=c0,
                        c1=c1)

# Divided line in the second stop
vrc_divided_line_1 = vrc(demand=demand,
                         travel_time=travel_time,
                         frequency=[10, 12],
                         divided_nodes=[1],
                         piv=piv,
                         pw=pw,
                         pr=pr,
                         board_alight_time=t,
                         c0=c0,
                         c1=c1)

# Divided line in the 10th stop
vrc_divided_line_10 = vrc(demand=demand,
                          travel_time=travel_time,
                          frequency=[12, 8],
                          divided_nodes=[9],
                          piv=piv,
                          pw=pw,
                          pr=pr,
                          board_alight_time=t,
                          c0=c0,
                          c1=c1)

print(f'VRC Fixed cost: {vrc_fix_cost}')
print(f'VRC Complete line: {vrc_complete_line}')
print(f'VRC Divided line on stop 1: {vrc_divided_line_1}')
print(f'VRC Divided line on stop 10: {vrc_divided_line_10}')

VRC Fixed cost: 76252.439312
VRC Complete line: 88787.19667062223
VRC Divided line on stop 1: 119434.78283688001
VRC Divided line on stop 10: 146492.8985046667


In [99]:
# Fixed cost
vrc_fix_cost = vrc_fix(demand=demand,
                       travel_time=travel_time,
                       piv=piv,
                       board_alight_time=t,
                       c0=c0,
                       c1=c1)

# Complete line
vrc_complete_line = vrc_optimization(demand=demand,
                                        travel_time=travel_time,
                                        divided_nodes=[],
                                        piv=piv,
                                        pw=pw,
                                        pr=pr,
                                        board_alight_time=t,
                                        c0=c0,
                                        c1=c1)

# Divided line in the second stop
vrc_divided_line_1 = vrc_optimization(demand=demand,
                                         travel_time=travel_time,
                                         divided_nodes=[1],
                                         piv=piv,
                                         pw=pw,
                                         pr=pr,
                                         board_alight_time=t,
                                         c0=c0,
                                         c1=c1)

# Divided line in the 10th stop
vrc_divided_line_10 = vrc_optimization(demand=demand,
                                          travel_time=travel_time,
                                          divided_nodes=[9],
                                          piv=piv,
                                          pw=pw,
                                          pr=pr,
                                          board_alight_time=t,
                                          c0=c0,
                                          c1=c1)

print(f'VRC Fixed cost: {vrc_fix_cost}')
print(f'VRC Complete line: {vrc_complete_line}')
print(f'VRC Divided line on stop 1: {vrc_divided_line_1}')
print(f'VRC Divided line on stop 10: {vrc_divided_line_10}')

VRC Fixed cost: 76252.439312
VRC Complete line:   message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -5283514.670164922
        x: [-2.598e+04]
      nit: 2
      jac: [ 2.066e+02]
 hess_inv: [[-2.634e+02]]
     nfev: 242
     njev: 115
VRC Divided line on stop 1:   message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: -530438880278537.2
        x: [-3.080e+13 -6.908e+03]
      nit: 10
      jac: [ 1.722e+01  0.000e+00]
 hess_inv: [[-3.484e+20 -7.813e+10]
            [-7.813e+10 -1.752e+01]]
     nfev: 1044
     njev: 344
VRC Divided line on stop 10:   message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -17893959670.105675
        x: [-1.092e+08 -5.501e+05]
      nit: 4
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 1.330e+01  1.174e-01]
            [ 1.174e-01  5.284e-03]]
     nfev: 54
     njev: 18


In [66]:
cost_operators(demand=demand,
               travel_time=travel_time,
               frequency=[12],
               board_alight_time=0.001,
               c0 = 0.1,
               c1 = 0.1,
               divided_nodes=[])

np.float64(23.856737268518515)